http://kdd.ics.uci.edu/databases/kddcup99/

In [1]:
! cp -rf /usr/lib/hive/conf/hive-site.xml /usr/lib/spark/conf/

In [2]:
from pyspark.sql import Row

In [3]:
max_k = 20
data_file = '/user/root/kddcup.data'

In [4]:
! rm -rf ./kddcup.data*

In [5]:
! wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz

--2021-11-03 15:14:07--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18115902 (17M) [application/x-gzip]
Saving to: ‘kddcup.data.gz’

kddcup.data.gz      100%[===================>]  17.28M  26.1MB/s    in 0.7s    

2021-11-03 15:14:08 (26.1 MB/s) - ‘kddcup.data.gz’ saved [18115902/18115902]



In [6]:
! gunzip kddcup.data.gz
! ls ./kddcup.data* -lh

-rw-r--r-- 1 root root 709M Jun 26  2007 ./kddcup.data


In [7]:
! head -3 ./kddcup.data

0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2,2,1.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,normal.


In [8]:
! wc -l kddcup.data

4898431 kddcup.data


In [9]:
! hdfs dfs -rm /user/root/kddcup.data
! hdfs dfs -rm -r /user/root/sample_standardized
! hdfs dfs -put kddcup.data /user/root

rm: `/user/root/kddcup.data': No such file or directory
rm: `/user/root/sample_standardized': No such file or directory


In [10]:
! hdfs dfs -ls /user/root

Found 2 items
drwxr-xr-x   - root hadoop          0 2021-11-03 15:13 /user/root/.sparkStaging
-rw-r--r--   2 root hadoop  742579829 2021-11-03 15:14 /user/root/kddcup.data


In [11]:
! wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names

--2021-11-03 15:14:30--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1307 (1.3K)
Saving to: ‘kddcup.names’

kddcup.names        100%[===================>]   1.28K  --.-KB/s    in 0s      

2021-11-03 15:14:30 (160 MB/s) - ‘kddcup.names’ saved [1307/1307]



In [12]:
! head -30 kddcup.names

back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,normal,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster.
duration: continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.


In [13]:
! hdfs dfs -rm /user/root/kddcup.names
! hdfs dfs -put kddcup.names /user/root

rm: `/user/root/kddcup.names': No such file or directory


In [14]:
! hdfs dfs -ls /user/root/

Found 3 items
drwxr-xr-x   - root hadoop          0 2021-11-03 15:13 /user/root/.sparkStaging
-rw-r--r--   2 root hadoop  742579829 2021-11-03 15:14 /user/root/kddcup.data
-rw-r--r--   2 root hadoop       1307 2021-11-03 15:14 /user/root/kddcup.names


In [15]:
fieldname_raw_rdd = sc.textFile('/user/root/kddcup.names')

In [16]:
all_attack = str(fieldname_raw_rdd.take(1)[0])

In [17]:
fieldname_rdd = fieldname_raw_rdd.\
filter(lambda x: x != all_attack).\
map(lambda y: y.split(':')).\
map(lambda z:(z[0],z[1]))

In [18]:
fieldname_rdd.collect()

[('duration', ' continuous.'),
 ('protocol_type', ' symbolic.'),
 ('service', ' symbolic.'),
 ('flag', ' symbolic.'),
 ('src_bytes', ' continuous.'),
 ('dst_bytes', ' continuous.'),
 ('land', ' symbolic.'),
 ('wrong_fragment', ' continuous.'),
 ('urgent', ' continuous.'),
 ('hot', ' continuous.'),
 ('num_failed_logins', ' continuous.'),
 ('logged_in', ' symbolic.'),
 ('num_compromised', ' continuous.'),
 ('root_shell', ' continuous.'),
 ('su_attempted', ' continuous.'),
 ('num_root', ' continuous.'),
 ('num_file_creations', ' continuous.'),
 ('num_shells', ' continuous.'),
 ('num_access_files', ' continuous.'),
 ('num_outbound_cmds', ' continuous.'),
 ('is_host_login', ' symbolic.'),
 ('is_guest_login', ' symbolic.'),
 ('count', ' continuous.'),
 ('srv_count', ' continuous.'),
 ('serror_rate', ' continuous.'),
 ('srv_serror_rate', ' continuous.'),
 ('rerror_rate', ' continuous.'),
 ('srv_rerror_rate', ' continuous.'),
 ('same_srv_rate', ' continuous.'),
 ('diff_srv_rate', ' continuous.

In [19]:
from pyspark.sql.types import *

In [20]:
def createschema(item):
    if item[1] == ' continuous.': 
        return StructField(item[0], FloatType(), True)
    if item[1] == ' symbolic.': 
        return StructField(item[0], StringType(), True)
    

In [21]:
fieldname_list = fieldname_rdd.map(createschema).collect()

In [22]:
fieldname_list.append(StructField('attack', StringType(), True))

In [23]:
fieldname_list

[StructField(duration,FloatType,true),
 StructField(protocol_type,StringType,true),
 StructField(service,StringType,true),
 StructField(flag,StringType,true),
 StructField(src_bytes,FloatType,true),
 StructField(dst_bytes,FloatType,true),
 StructField(land,StringType,true),
 StructField(wrong_fragment,FloatType,true),
 StructField(urgent,FloatType,true),
 StructField(hot,FloatType,true),
 StructField(num_failed_logins,FloatType,true),
 StructField(logged_in,StringType,true),
 StructField(num_compromised,FloatType,true),
 StructField(root_shell,FloatType,true),
 StructField(su_attempted,FloatType,true),
 StructField(num_root,FloatType,true),
 StructField(num_file_creations,FloatType,true),
 StructField(num_shells,FloatType,true),
 StructField(num_access_files,FloatType,true),
 StructField(num_outbound_cmds,FloatType,true),
 StructField(is_host_login,StringType,true),
 StructField(is_guest_login,StringType,true),
 StructField(count,FloatType,true),
 StructField(srv_count,FloatType,true),

In [24]:
schema = StructType(fieldname_list)

In [25]:
schema

StructType(List(StructField(duration,FloatType,true),StructField(protocol_type,StringType,true),StructField(service,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,FloatType,true),StructField(dst_bytes,FloatType,true),StructField(land,StringType,true),StructField(wrong_fragment,FloatType,true),StructField(urgent,FloatType,true),StructField(hot,FloatType,true),StructField(num_failed_logins,FloatType,true),StructField(logged_in,StringType,true),StructField(num_compromised,FloatType,true),StructField(root_shell,FloatType,true),StructField(su_attempted,FloatType,true),StructField(num_root,FloatType,true),StructField(num_file_creations,FloatType,true),StructField(num_shells,FloatType,true),StructField(num_access_files,FloatType,true),StructField(num_outbound_cmds,FloatType,true),StructField(is_host_login,StringType,true),StructField(is_guest_login,StringType,true),StructField(count,FloatType,true),StructField(srv_count,FloatType,true),StructField(serror_rate,FloatTy

In [26]:
! pip install numpy pandas

In [27]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler,\
OneHotEncoder
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,\
BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier,\
NaiveBayes, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [28]:
raw_df = sqlContext.read.format('com.databricks.spark.csv').\
options(header='false', inferschema='true').\
schema(schema).load('/user/root/kddcup.data')

In [29]:
raw_df1 = raw_df.repartition(60)

In [30]:
raw_df2 = raw_df1.cache()

In [31]:
raw_df2.printSchema()

root
 |-- duration: float (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: float (nullable = true)
 |-- dst_bytes: float (nullable = true)
 |-- land: string (nullable = true)
 |-- wrong_fragment: float (nullable = true)
 |-- urgent: float (nullable = true)
 |-- hot: float (nullable = true)
 |-- num_failed_logins: float (nullable = true)
 |-- logged_in: string (nullable = true)
 |-- num_compromised: float (nullable = true)
 |-- root_shell: float (nullable = true)
 |-- su_attempted: float (nullable = true)
 |-- num_root: float (nullable = true)
 |-- num_file_creations: float (nullable = true)
 |-- num_shells: float (nullable = true)
 |-- num_access_files: float (nullable = true)
 |-- num_outbound_cmds: float (nullable = true)
 |-- is_host_login: string (nullable = true)
 |-- is_guest_login: string (nullable = true)
 |-- count: float (nullable = true)
 |-- srv_count: float (nullable = 

In [32]:
import pandas as pd
pd.DataFrame(raw_df2.take(5), \
             columns=raw_df2.columns).transpose()

,0,1,2,3,4
duration,0,0,0,0,0
protocol_type,udp,tcp,tcp,tcp,tcp
service,domain_u,ftp_data,private,private,http
flag,SF,SF,S0,S0,REJ
src_bytes,42,748,0,0,0
dst_bytes,42,0,0,0,0
land,0,0,0,0,0
wrong_fragment,0,0,0,0,0
urgent,0,0,0,0,0
hot,0,0,0,0,0


In [33]:
raw_df2.count()

4898431

In [34]:
analysis1_df = raw_df2.\
groupBy('attack').count()

In [35]:
analysis1_df.orderBy(analysis1_df['count'],\
                     ascending = False).show()

+----------------+-------+
|          attack|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
+----------------+-------+
only showing top 20 rows



In [36]:
nolabel_df = raw_df2.drop('attack').drop('service')

In [37]:
nolabel_df.count()

4898431

In [38]:
nolabel_df.dropna().count()

4898431

In [39]:
nolabel_df.take(1)

[Row(duration=0.0, protocol_type='udp', flag='SF', src_bytes=42.0, dst_bytes=42.0, land='0', wrong_fragment=0.0, urgent=0.0, hot=0.0, num_failed_logins=0.0, logged_in='0', num_compromised=0.0, root_shell=0.0, su_attempted=0.0, num_root=0.0, num_file_creations=0.0, num_shells=0.0, num_access_files=0.0, num_outbound_cmds=0.0, is_host_login='0', is_guest_login='0', count=6.0, srv_count=12.0, serror_rate=0.0, srv_serror_rate=0.0, rerror_rate=0.0, srv_rerror_rate=0.0, same_srv_rate=1.0, diff_srv_rate=0.0, srv_diff_host_rate=0.25, dst_host_count=255.0, dst_host_srv_count=239.0, dst_host_same_srv_rate=0.9399999976158142, dst_host_diff_srv_rate=0.009999999776482582, dst_host_same_src_port_rate=0.0, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=0.0, dst_host_srv_serror_rate=0.0, dst_host_rerror_rate=0.0, dst_host_srv_rerror_rate=0.0)]

In [40]:
nolabel_df.columns

['duration',
 'protocol_type',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate']

In [41]:
protocol_typeIndexer = StringIndexer(inputCol="protocol_type", \
                                     outputCol="protocol_typeIndex")
protocol_typeVector = OneHotEncoder(inputCol='protocol_typeIndex', \
                                    outputCol='protocol_typeVec')

flagIndexer = StringIndexer(inputCol="flag", outputCol="flagIndex")
flagVector = OneHotEncoder(inputCol='flagIndex', outputCol='flagVec')

In [42]:
assembler = VectorAssembler\
(inputCols=['duration', 'dst_bytes','wrong_fragment', 'urgent', \
            'hot', 'num_failed_logins', 'num_compromised', 'root_shell', \
            'su_attempted', 'num_root', 'num_file_creations', 'num_shells', \
            'num_access_files', 'num_outbound_cmds', 'count', 'srv_count', \
            'serror_rate', 'srv_serror_rate', 'rerror_rate', \
            'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', \
            'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', \
            'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', \
            'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', \
            'dst_host_serror_rate', 'dst_host_srv_serror_rate', \
            'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',\
            'protocol_typeVec','flagVec'],outputCol='features')

In [43]:
kmeans = KMeans(featuresCol='features',\
                k=20,predictionCol='prediction',\
                maxIter=20,initMode="random", \
                initSteps=5)

In [44]:
pipeline = Pipeline().\
setStages([protocol_typeIndexer,protocol_typeVector,\
           flagIndexer,flagVector,assembler,kmeans])

In [45]:
kMeansPredictionModel = pipeline.fit(nolabel_df)

In [46]:
predictionResult = kMeansPredictionModel.transform(raw_df2)

In [47]:
result=predictionResult.groupBy('attack','prediction').count()

In [48]:
result.orderBy(result['prediction'],ascending=False).show(100)

+----------------+----------+-------+
|          attack|prediction|  count|
+----------------+----------+-------+
|          smurf.|         9| 104121|
|          satan.|         9|  12641|
|         normal.|         9|   2505|
|           nmap.|         9|      1|
|    warezmaster.|         8|     15|
|         normal.|         8|     79|
|      portsweep.|         7|   9520|
|           imap.|         7|     10|
|      ftp_write.|         7|      7|
|           perl.|         7|      3|
|    warezclient.|         7|    998|
|          satan.|         7|   3250|
|buffer_overflow.|         7|     21|
|         normal.|         7| 827935|
|            spy.|         7|      2|
|   guess_passwd.|         7|     53|
|           land.|         7|     21|
|           back.|         7|     25|
|       multihop.|         7|      3|
|            pod.|         7|    264|
|        ipsweep.|         7|  12480|
|          smurf.|         7|   1609|
|        rootkit.|         7|      7|
|    warezma

In [65]:
result.filter(result['prediction'] == '1').\
orderBy(result['prediction'],ascending = False).show(150)

+-------+----------+-------+
| attack|prediction|  count|
+-------+----------+-------+
| smurf.|         1|2615673|
|normal.|         1|      9|
+-------+----------+-------+



In [50]:
#kMeansPredictionModel.save('gs://aekanuntest/cluster_best_smurf')

In [51]:
! ls -l corrected*

ls: cannot access 'corrected*': No such file or directory


In [52]:
! rm -rf corrected*

In [53]:
! wget http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz

--2021-11-03 15:17:42--  http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1409035 (1.3M) [application/x-gzip]
Saving to: ‘corrected.gz’

corrected.gz        100%[===================>]   1.34M  3.33MB/s    in 0.4s    

2021-11-03 15:17:42 (3.33 MB/s) - ‘corrected.gz’ saved [1409035/1409035]



In [54]:
! gunzip corrected.gz

In [55]:
! ls -l corrected*

-rw-r--r-- 1 root root 47251540 May 16  2000 corrected


In [56]:
! head -3 corrected

0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.
0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.
0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.


In [57]:
! hdfs dfs -put corrected /user/root/

In [58]:
test_df = sqlContext.read.format('com.databricks.spark.csv').\
options(header='false', inferschema='true').schema(schema).\
load('/user/root/corrected')

In [59]:
test_df1 = test_df.repartition(60)

In [60]:
test_df2 = test_df1.cache()

In [61]:
test_df2.count()

311029

In [62]:
test_result = kMeansPredictionModel.\
transform(test_df2)

In [63]:
final_result = test_result.\
groupBy('attack','prediction').count()

In [66]:
final_result.filter(final_result['prediction'] == '1').\
orderBy(final_result['prediction'],ascending = False).show(150)

+--------+----------+------+
|  attack|prediction| count|
+--------+----------+------+
| normal.|         1|   586|
|ipsweep.|         1|     3|
|  smurf.|         1|131102|
+--------+----------+------+

